In [29]:
import mat73
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [30]:
data_dict_female = mat73.loadmat("../data/Emotrans1_girl_data_preprocessed.mat", use_attrdict=True)
arr_f = np.array(data_dict_female["All_Feature"])

In [31]:
data_dict = mat73.loadmat("../data/Emotrans1_Boy_data_preprocessed.mat", use_attrdict=True)
arr_m = np.array(data_dict["All_Feature"])

In [32]:
#retrive pos or neg return np 
def get_pos_or_neg(data,position):
    arr = []
    index = 0
    for i in range(data.shape[0]):
        temp = []
        for j in position:
            for k in data[i][j[0]][j[1]]:
                temp.append(k)
        arr.append(temp)
    return np.array(arr)

In [41]:
# select frequenceis and (0-4s -> 0 or 0.5-4.5s -> 1)
def get_selected(data=None,freq = None, sec=None):
    if freq:
        data = np.delete(data,freq,axis=3)
    if sec == 0:
        return np.delete(data, 0, axis=4)
    elif sec == 1:
        return np.delete(data,1,axis=4)
    else:
        print('sec not specified')
        return data 

In [42]:
# female data combined 
pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]]

pos_df_f = get_pos_or_neg(arr_f,pos)
neg_df_f = get_pos_or_neg(arr_f,neg)

pos_df_f=get_selected(pos_df_f,sec=1).reshape(15,96,128,8)
neg_df_f = get_selected(neg_df_f,sec=1).reshape(15,96,128,8)
# raw_data_f = np.concatenate((pos_df_f,neg_df_f),axis=1).reshape(15,192,128*8)
# y_f = np.concatenate((np.ones((15,96)),np.zeros((15,96))),axis=1)


In [43]:
pos_df_m = get_pos_or_neg(arr_m,pos)
neg_df_m = get_pos_or_neg(arr_m,neg)
pos_df_m = get_selected(pos_df_m,sec=1).reshape(18,96,128,8)
neg_df_m = get_selected(neg_df_m,sec=1).reshape(18,96,128,8)
# raw_data_m = np.concatenate((pos_df_m,neg_df_m),axis=1).reshape(18,192,128*8)
# y_m = np.concatenate((np.ones((18,96)),np.zeros((18,96))),axis=1)


In [44]:
raw_data_p = np.concatenate((pos_df_m,pos_df_f),axis=0).reshape(33,96,128*8)
raw_data_n = np.concatenate((neg_df_f,neg_df_m),axis=0).reshape(33,96,128*8)
y = np.concatenate((np.ones((33,96)),np.zeros((33,96))),axis=1)
print(raw_data_p.shape)
print(raw_data_n.shape)
print(y.shape)

(33, 96, 1024)
(33, 96, 1024)
(33, 192)


In [45]:
raw_data = np.append(raw_data_p,raw_data_n,axis=1)
print(raw_data.shape)
print(y.shape)
# print(y.shape)

(33, 192, 1024)
(33, 192)


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
iter = 0
alphas = [0.1,0.01,0.001]
coefs = []

iter = 0
train_scores=[]
test_scores = []
for i in range(32):
    model = RandomForestClassifier(n_estimators=500, # Number of trees to train
                               criterion='gini', # How to train the trees. Also supports entropy.
                               max_depth=None, # Max depth of the trees. Not necessary to change.
                               min_samples_split=2, # Minimum samples to create a split.
                               min_samples_leaf=0.001, # Minimum samples in a leaf. Accepts fractions for %. This is 0.1% of sample.
                               min_weight_fraction_leaf=0.0, # Same as above, but uses the class weights.
                               max_features='auto', # Maximum number of features per split (not tree!) by default is sqrt(vars)
                               max_leaf_nodes=None, # Maximum number of nodes.
                               min_impurity_decrease=0.0001, # Minimum impurity decrease. This is 10^-3.
                               bootstrap=True, # If sample with repetition. For large samples (>100.000) set to false.
                               oob_score=True,  # If report accuracy with non-selected cases.
                               n_jobs=-1, # Parallel processing. Set to -1 for all cores. Watch your RAM!!
                               random_state=20190305, # Seed
                               verbose=1, # If to give info during training. Set to 0 for silent training.
                               warm_start=False, # If train over previously trained tree.
                               class_weight='balanced'
                                            )
    display(iter)
    iter+=1
    x_train,x_test = np.concatenate((raw_data[:i],raw_data[i+1:])).reshape(32*192,1024),raw_data[i].reshape(192,1024)
    y_train,y_test = np.concatenate((y[:i],y[i+1:])).reshape(32*192),y[i].reshape(192)
    x_train,y_train = shuffle(x_train,y_train)
    model.fit(x_train, y_train)
    # training loss
    pred_train_lasso= model.predict(x_train)
    temp = accuracy_score(y_train,pred_train_lasso)
    display("training accuracy: ",temp)
    train_scores.append(temp)

    #testing 
    pred_test_lasso= model.predict(x_test)
    temp = accuracy_score(y_test,pred_test_lasso)
    display("testing accuracy: ",temp)
    test_scores.append(temp)

    coefs.append(model.feature_importances_)
np.savetxt("combined_training_score_rf.csv", np.array(train_scores), delimiter=",")
np.savetxt("combined_test_score_rf.csv", np.array(test_scores), delimiter=",")
np.savetxt("combined_coefs_rf.csv", np.array(coefs), delimiter=",")

0

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    8.8s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    0.2s finished


'training accuracy: '

0.9851888020833334

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    0.1s finished


'testing accuracy: '

0.15625

1

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [28]:
raw_data.shape

(66, 96, 1024)

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
iter = 0
alphas = [0.1,0.01,0.001]
coefs = []

iter = 0
train_scores=[]
test_scores = []
for i in range(32):
    rf = XGBClassifier(max_depth=3,                 # Depth of each tree
                                learning_rate=0.1,            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                                n_estimators=100,             # How many trees to use, the more the better, but decrease learning rate if many used.
                                verbosity=1,                  # If to show more errors or not.
                                objective='binary:logistic',  # Type of target variable.
                                booster='gbtree',             # What to boost. Trees in this case.
                                n_jobs=2,                     # Parallel jobs to run. Set your processor number.
                                gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                                subsample=1,                  # Subsample ratio. Can set lower
                                colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                                colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest.
                                colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                                reg_alpha=1,                  # Regularizer for first fit. alpha = 1, lambda = 0 is LASSO.
                                reg_lambda=0,                 # Regularizer for first fit.
                                scale_pos_weight=1,           # Balancing of positive and negative weights.
                                base_score=0.5,               # Global bias. Set to average of the target rate.
                                random_state=20201107,        # Seed
                                missing=None                  # How are nulls encoded?
                                )
    display(iter)
    iter+=1
    x_train,x_test = np.concatenate((raw_data[:i],raw_data[i+1:])).reshape(32*192,1024),raw_data[i].reshape(192,1024)
    y_train,y_test = np.concatenate((y[:i],y[i+1:])).reshape(32*192),y[i].reshape(192)
    x_train,y_train = shuffle(x_train,y_train)
    model.fit(x_train, y_train)
    # training loss
    pred_train_lasso= model.predict(x_train)
    temp = accuracy_score(y_train,pred_train_lasso)
    display("training accuracy: ",temp)
    train_scores.append(temp)

    #testing 
    pred_test_lasso= model.predict(x_test)
    temp = accuracy_score(y_test,pred_test_lasso)
    display("testing accuracy: ",temp)
    test_scores.append(temp)

    coefs.append(model.feature_importances_)
np.savetxt("combined_training_score.csv", np.array(train_scores), delimiter=",")
np.savetxt("combined_test_score.csv", np.array(test_scores), delimiter=",")
np.savetxt("combined_coefs.csv", np.array(coefs), delimiter=",")

0

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    9.4s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    0.2s finished


'training accuracy: '

0.9881184895833334

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    0.1s finished


'testing accuracy: '

0.13020833333333334

1

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    3.3s


KeyboardInterrupt: 